In [7]:
import numpy as np
df=np.load("/Users/rohitviswam/Downloads/CSI-02-0005/H_32T4R_30_1RB.npy")
data=df.reshape(2100,398,256)
data.shape
feature_len=data.shape[-1]

In [8]:
data.shape

(2100, 398, 256)

In [9]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [10]:
data = data.reshape(data.shape[0], -1)  # shape becomes (2100, 398*256)


In [17]:
data.shape

(2100, 101888)

In [18]:
import numpy as np

# Load your dataset if it's not already loaded
# data = np.load('path/to/your/data.npy')

# Set up parameters
num_UEs, num_timesteps = data.shape
sequence_length = 10  # Use the first 10 values as input
stride = 1  # Moving one step at a time

# Initialize lists to hold input and output sequences
X, y = [], []

# Loop through each UE
for ue_idx in range(num_UEs):
    ue_data = data[ue_idx]
    for start_idx in range(0, num_timesteps - sequence_length, stride):
        # Extract the input and output sequence
        end_idx = start_idx + sequence_length
        input_seq = ue_data[start_idx:end_idx]
        output_value = ue_data[end_idx]
        
        # Append to the lists
        X.append(input_seq)
        y.append(output_value)

# Convert lists to NumPy arrays for easier processing
X = np.array(X)
y = np.array(y)

print(f"Input shape: {X.shape}")
print(f"Output shape: {y.shape}")


Input shape: (213943800, 10)
Output shape: (213943800,)


In [19]:
# Define the train-test split ratio (e.g., 80% train, 20% test)
train_ratio = 0.8
train_size = int(len(X) * train_ratio)

# Split the data into training and testing while keeping the time series intact
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Check the resulting shapes
print(f"Training data shapes: X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Testing data shapes: X_test: {X_test.shape}, y_test: {y_test.shape}")

Training data shapes: X_train: (171155040, 10), y_train: (171155040,)
Testing data shapes: X_test: (42788760, 10), y_test: (42788760,)


In [28]:
import numpy as np
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv1D, BatchNormalization, ReLU, Dropout, Add, Dense, Flatten
from tensorflow.keras.optimizers import Adam

# TCN residual block
def residual_block(x, dilation_rate, num_filters, kernel_size, dropout_rate=0.0):
    conv1 = Conv1D(num_filters, kernel_size, padding='causal', dilation_rate=dilation_rate)(x)
    bn1 = BatchNormalization()(conv1)
    act1 = ReLU()(bn1)
    drop1 = Dropout(dropout_rate)(act1)

    conv2 = Conv1D(num_filters, kernel_size, padding='causal', dilation_rate=dilation_rate)(drop1)
    bn2 = BatchNormalization()(conv2)
    act2 = ReLU()(bn2)
    drop2 = Dropout(dropout_rate)(act2)

    if x.shape[-1] != num_filters:
        x = Conv1D(num_filters, kernel_size=1)(x)

    out = Add()([x, drop2])
    out = ReLU()(out)
    return out

# Build TCN model
def build_tcn_model(input_shape, num_blocks=3, num_filters=64, kernel_size=3, dropout_rate=0.2):
    inputs = Input(shape=input_shape)
    x = inputs

    for i in range(num_blocks):
        dilation_rate = 2 ** i
        x = residual_block(x, dilation_rate, num_filters, kernel_size, dropout_rate)

    x = Flatten()(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Example usage
input_length = 10  # The number of previous time steps used for input
num_features = 1  # Modify according to your dataset

# Adjust `num_features` if your input sequences contain multiple features.
model = build_tcn_model(input_shape=(input_length, num_features))

model.summary()

model.save('best_tcn_model.keras')


Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 10, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_56 (Conv1D)  │ (None, 10, 64)    │        256 │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 64)    │        256 │ conv1d_56[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_72 (ReLU)     │ (None, 10, 64)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_48          │ (None, 10, 64)    │          0 │ re_lu_72[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_57 (Conv1D)  │ (None, 10, 64)    │     12,352 │ dropout_48[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 64)    │        256 │ conv1d_57[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_73 (ReLU)     │ (None, 10, 64)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_58 (Conv1D)  │ (None, 10, 64)    │        128 │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_49          │ (None, 10, 64)    │          0 │ re_lu_73[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_24 (Add)        │ (None, 10, 64)    │          0 │ conv1d_58[0][0],  │
│                     │                   │            │ dropout_49[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_74 (ReLU)     │ (None, 10, 64)    │          0 │ add_24[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_59 (Conv1D)  │ (None, 10, 64)    │     12,352 │ re_lu_74[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 64)    │        256 │ conv1d_59[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_75 (ReLU)     │ (None, 10, 64)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_50          │ (None, 10, 64)    │          0 │ re_lu_75[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_60 (Conv1D)  │ (None, 10, 64)    │     12,352 │ dropout_50[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 64)    │        256 │ conv1d_60[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_76 (ReLU)     │ (None, 10, 64)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_51          │ (None, 10, 64)    │          0 │ re_lu_76[0][0]    │
│ (Dropout)           │                   │            │                 

 Total params: 64,321 (251.25 KB)

 Trainable params: 63,553 (248.25 KB)

 Non-trainable params: 768 (3.00 KB)

In [30]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Assuming you have `X_train`, `y_train`, `X_test`, and `y_test` arrays loaded from preprocessing
# Make sure the input data is reshaped to have the expected dimensions (num_samples, sequence_length, num_features)

# Reshape input to have the required dimensions: (samples, input_length, num_features)
num_features = 1  # Adjust based on the dataset (e.g., 1 for univariate, multiple if needed)
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], num_features))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], num_features))

# Initialize the TCN model
model = build_tcn_model(input_shape=(X_train.shape[1], num_features))

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_tcn_model.keras', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    X_train_reshaped, y_train,
    validation_data=(X_test_reshaped, y_test),
    epochs=5,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]
)



Epoch 1/5
  42658/5348595 ━━━━━━━━━━━━━━━━━━━━ 13:39:09 9ms/step - loss: 0.5068

In [ ]:
# Check the training history for loss/accuracy improvement
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss History')
plt.show()
